# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [68]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [5]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2521177204571,
 'week52high': 148.89,
 'week52low': 90.57,
 'week52highSplitAdjustOnly': 146.34,
 'week52lowSplitAdjustOnly': 92.99,
 'week52change': 0,
 'sharesOutstanding': 17115641183,
 'float': 0,
 'avg10Volume': 80373369,
 'avg30Volume': 76720885,
 'day200MovingAvg': 132.48,
 'day50MovingAvg': 134.2,
 'employees': 149252,
 'ttmEPS': 4.62,
 'ttmDividendRate': 0.8652702826263878,
 'dividendYield': 0.005770376922689444,
 'nextDividendDate': '',
 'exDividendDate': '2021-05-05',
 'nextEarningsDate': '2021-07-17',
 'peRatio': 32.77959350120395,
 'beta': 1.632478883768031,
 'maxChangePercent': 56.55824117092863,
 'year5ChangePercent': 5.71419998123302,
 'year2ChangePercent': 1.9641734816179313,
 'year1ChangePercent': 0.5657176516331971,
 'ytdChangePercent': 0,
 'month6ChangePercent': 0.112840708512663,
 'month3ChangePercent': 0.13875958461602098,
 'month1ChangePercent': 0.15168193366795796,
 'day30ChangePercent': 0.15050786871441407,
 'day5Chang

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [6]:
data['year1ChangePercent']

0.5657176516331971

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [8]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [23]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['price'],
            data[symbol]['stats']['year1ChangePercent'],
            'N/A'
        ],
        index=my_columns),
            ignore_index=True
        )
        
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,150.080,0.708086,N/A
1,AAL,20.461,0.7384,N/A
2,AAP,209.560,0.556043,N/A
3,AAPL,145.168,0.585237,N/A
4,ABBV,119.380,0.23813,N/A
...,...,...,...,...
500,YUM,120.937,0.395422,N/A
501,ZBH,166.500,0.398612,N/A
502,ZBRA,554.600,1.20718,N/A
503,ZION,52.200,0.710745,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [25]:
final_dataframe.sort_values('One-Year Price Return', ascending=False, inplace=True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe

/home/bobbyhuck/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,73.060,3.87458,N/A
1,441,TPR,40.810,2.2882,N/A
2,179,FCX,35.600,1.93816,N/A
3,148,DVN,28.600,1.80847,N/A
4,410,SIVB,553.830,1.69849,N/A
5,106,COF,153.300,1.68189,N/A
6,208,GPS,31.880,1.66595,N/A
7,253,IVZ,26.330,1.64049,N/A
8,272,KSS,51.085,1.61901,N/A
9,149,DXC,41.000,1.57334,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [28]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio')
    
    try:
        float(portfolio_size)
    except ValueError:
        print('That is not a number! \nPlease try again:')
        portfolio_size = input('Enter the size of your portfolio:')
        
portfolio_input()
print(portfolio_size)

Enter the size of your portfolio 10000000


10000000


In [33]:
position_size = float(portfolio_size) / len(final_dataframe.index)

for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i, 'Price'])
    
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,73.060,3.87458,2737
1,441,TPR,40.810,2.2882,4900
2,179,FCX,35.600,1.93816,5617
3,148,DVN,28.600,1.80847,6993
4,410,SIVB,553.830,1.69849,361
5,106,COF,153.300,1.68189,1304
6,208,GPS,31.880,1.66595,6273
7,253,IVZ,26.330,1.64049,7595
8,272,KSS,51.085,1.61901,3915
9,149,DXC,41.000,1.57334,4878


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [79]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile'
]

hqm_dataframe = pd.DataFrame(columns=hqm_columns)

for symbol_string in symbol_strings:
        batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
        data = requests.get(batch_api_call_url).json()
        
        for symbol in symbol_string.split(','):
            hqm_dataframe = hqm_dataframe.append(
                pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    'N/A',
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month6ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month3ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month1ChangePercent'],
                    'N/A'
                ],
                index=hqm_columns),
                ignore_index=True
            )
            
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile
0,A,150.560,N/A,0.699573,N/A,0.192514,N/A,0.164883,N/A,0.0915403,N/A
1,AAL,20.669,N/A,0.740212,N/A,0.328315,N/A,-0.155964,N/A,-0.168724,N/A
2,AAP,215.830,N/A,0.568234,N/A,0.262528,N/A,0.154976,N/A,0.0931758,N/A
3,AAPL,143.720,N/A,0.580076,N/A,0.109713,N/A,0.135788,N/A,0.148377,N/A
4,ABBV,116.820,N/A,0.239183,N/A,0.120653,N/A,0.128108,N/A,0.0345881,N/A
...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,117.420,N/A,0.387266,N/A,0.13096,N/A,0.0384325,N/A,-0.00435109,N/A
501,ZBH,165.210,N/A,0.403217,N/A,0.0143385,N/A,-0.00891404,N/A,0.022204,N/A
502,ZBRA,554.290,N/A,1.1867,N/A,0.352281,N/A,0.130508,N/A,0.0836209,N/A
503,ZION,51.657,N/A,0.694393,N/A,0.0400528,N/A,-0.0719226,N/A,-0.113535,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [80]:
hqm_dataframe.shape

(505, 11)

In [81]:
rows_to_drop = [118, 165, 326, 327]

hqm_dataframe.drop(rows_to_drop, inplace=True)

hqm_dataframe.shape

(501, 11)

In [82]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])
        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile
0,A,150.560,N/A,0.699573,73.8523,0.192514,61.8762,0.164883,86.8263,0.0915403,88.4232
1,AAL,20.669,N/A,0.740212,77.4451,0.328315,86.0279,-0.155964,2.39521,-0.168724,0.998004
2,AAP,215.830,N/A,0.568234,61.2774,0.262528,75.4491,0.154976,84.6307,0.0931758,89.022
3,AAPL,143.720,N/A,0.580076,62.0758,0.109713,40.7186,0.135788,79.8403,0.148377,95.6088
4,ABBV,116.820,N/A,0.239183,22.5549,0.120653,43.1138,0.128108,77.4451,0.0345881,72.6547
...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,117.420,N/A,0.387266,40.3194,0.13096,45.3094,0.0384325,48.1038,-0.00435109,52.8942
501,ZBH,165.210,N/A,0.403217,41.7166,0.0143385,14.7705,-0.00891404,27.3453,0.022204,67.2655
502,ZBRA,554.290,N/A,1.1867,94.012,0.352281,88.024,0.130508,78.0439,0.0836209,86.4271
503,ZION,51.657,N/A,0.694393,73.2535,0.0400528,19.5609,-0.0719226,9.38124,-0.113535,6.98603


In [63]:
# We had 4 stocks that did not have values for '{Time-Period} Price Return'
# val of None populated these cols and threw an error when calculating Percentiles
np.where(hqm_dataframe.isnull() == True) 

(array([118, 118, 118, 118, 165, 165, 165, 165, 326, 326, 326, 326, 327,
        327, 327, 327]),
 array([3, 5, 7, 9, 3, 5, 7, 9, 3, 5, 7, 9, 3, 5, 7, 9]))

In [62]:
print(hqm_dataframe.loc[118,:], hqm_dataframe.loc[165,:], hqm_dataframe.loc[326,:], hqm_dataframe.loc[327,:])

Ticker                            CTL
Price                              11
Number of Shares to Buy           N/A
One-Year Price Return            None
One-Year Return Percentile          0
Six-Month Price Return           None
Six-Month Return Percentile         0
Three-Month Price Return         None
Three-Month Return Percentile       0
One-Month Price Return           None
One-Month Return Percentile         0
Name: 118, dtype: object Ticker                           ETFC
Price                            51.7
Number of Shares to Buy           N/A
One-Year Price Return            None
One-Year Return Percentile          0
Six-Month Price Return           None
Six-Month Return Percentile         0
Three-Month Price Return         None
Three-Month Return Percentile       0
One-Month Price Return           None
One-Month Return Percentile         0
Name: 165, dtype: object Ticker                             MYL
Price                            16.12
Number of Shares to Buy            N

In [60]:
hqm_dataframe.loc[1,:]

Ticker                                AAL
Price                              20.352
Number of Shares to Buy               N/A
One-Year Price Return            0.718782
One-Year Return Percentile              0
Six-Month Price Return           0.330681
Six-Month Return Percentile             0
Three-Month Price Return        -0.156475
Three-Month Return Percentile           0
One-Month Price Return          -0.167109
One-Month Return Percentile             0
Name: 1, dtype: object

## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [ ]:
from statistics import mean

for row

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

## Saving Our Excel Output

As before, saving our Excel output is very easy: